# DistilBERT - 실습
> fine-tuning으로 감정 분석 모델 학습하기

- pre-trained DistilBERT를 감정 분석 문제에 적용

### DIstilBERT 구조 도식화
```
입력 예시: 문장 2개 → 배치 처리
====================================================
1. Raw Text:
   ["this is great", "totally boring..."]

2. Tokenizer 처리:
   → input_ids:         [[101, 2023, 2003, 2307, 102, 0, 0],  
                          [101,  totally, boring, ..., 102, 0]]
   → attention_mask:     [[  1,    1,    1,    1,   1, 0, 0],
                          [  1,    1,    1,   ...,  1, 0]]
   → shape: [batch_size, seq_len] = [2, 7]

====================================================
3. DistilBERT 모델 입력:
   model(input_ids, attention_mask)

4. 출력 구조:
   outputs = model(...)
   └── outputs.last_hidden_state → shape: [2, 7, 768]

     📌 의미:
     - 각 토큰(7개)에 대해 768차원의 벡터가 출력됨
     - 문장마다 7개의 토큰 벡터 존재
     - 이 벡터들은 context-aware (예: 'great'의 벡터는 'this is'와 'great' 문맥)

5. 분류 태스크:
   보통 이 중 첫 번째 벡터, 즉 [CLS] 벡터를 사용
   → x = outputs.last_hidden_state[:, 0, :]  → shape: [2, 768]
   → classifier(x) → shape: [2, num_classes]

====================================================
```

### Packages
- `datasets`: HuggingFace의 데이터셋 로딩용 라이브러리 (IMDB, AG_News 등)
- `sentencepiece`, `sacremoses`: 일부 tokenizer에서 사용하는 전처리 도구
- 그 외 유틸 (`tqdm`: 진행바, `requests`: HTTP 요청 등)

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

### Libraries
- `load_dataset`: HuggingFace 데이터셋을 쉽게 불러오기 위한 함수

### Tokenizer
- `tokenizer`: DistilBERT용 토크나이저
  - `distilbert-base-uncased`: 소문자만 사용하는 경량 BERT 모델

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# DistilBERT 모델용 tokenizer 로드 (pretrained)
# 이 tokenizer는 문장을 토큰화해서 모델이 이해할 수 있는 input_ids로 변환해줌
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

### 데이터셋 불러오기(`load_dataset`) 및 `collate_fn`정의
**데이터셋**   
- **IMDB**(binary classification -> 0: 부정 / 1: 긍정)
- `[:5%]`: train/test 각각 5%만 샘플링하여 사용   

<br>

**`collate_fn`**   
```
batch 단위로 text, label 추출
- text: token화 한 input_ids PyTorch LongTensor
- label: 기존 label PyTorch LongTensor

attention mask는 사용하지 않음
- self-attention 시 마스킹 처리를 하지 않음
- 모든 토큰에 대해 유효 단어로 처리하여 padding token까지 학습세 사용하게 됨

=> 학습 성능이 하향될 수 있음
```
- `max_len`: 400
- `batch_size`: 64
- `padding`, `truncation`: True

<br>

**DistilBERT 흐름 시각화**
```
          ┌─────────────────────┐
          │  input_ids          │
          │  attention_mask     │
          └─────────────────────┘
                    │
                    ▼
          ┌─────────────────────┐
          │    DistilBERT        │
          │  (transformer layers)│
          └─────────────────────┘
                    │
                    ▼
┌────────────────────────────────────────────┐
│ last_hidden_state (shape: [B, L, 768])     │
└────────────────────────────────────────────┘
                    │
                    ▼
         [CLS] token vector만 추출 → shape: [B, 768]
                    │
                    ▼
         Linear → num_classes 출력 → [B, C]
```

In [ ]:
# IMDB 감정 분석 데이터셋의 5%만 로드 (학습 데이터와 테스트 데이터 각각)
train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

# 데이터를 배치로 묶기 위한 함수 정의
def collate_fn(batch):
    max_len = 400  # 입력 문장의 최대 길이 설정
    texts, labels = [], []  # 입력 문장들과 라벨들을 저장할 리스트

    # 배치 내 각 샘플에 대해 text와 label 추출
    for row in batch:
        labels.append(row['label'])
        texts.append(row['text'])

    # tokenizer로 텍스트를 토큰화하고, 최대 길이로 패딩 및 자르기
    # tokenizer는 사전에 정의되어 있어야 함 (예: tokenizer = AutoTokenizer.from_pretrained(...))
    texts = torch.LongTensor(
        tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids
    )

    # 라벨 리스트를 LongTensor로 변환
    labels = torch.LongTensor(labels)

    # 모델 학습에 필요한 입력 (토큰화된 문장들)과 정답 라벨 반환
    return texts, labels

# 학습용 DataLoader 정의 (shuffle=True로 배치 순서 랜덤화)
train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)

# 테스트용 DataLoader 정의 (shuffle=False로 배치 순서 고정)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

### DistilBERT 모델 로드
- HuggingFace에서 pre-trained된 DistilBERT 모델을 PyTorch Hub로 로드
- **`input`**: tokenizing된 데이터의 **input_ids만**
- **`output`**: `last_hidden_state`

In [4]:
#DistilBERT 모델을 PyTorch Hub에서 로드 후 model 출력
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

### Classifier 모델 정의
```
DistilBERT의 [CLS] 토큰 벡터(x[:,0])를 추출하여 classification task 수행
```
- binary classification: `nn.Linear(768, 1)`
- sigmoid, softmax등의 확률화 함수 없이 raw logit 값을 출력

In [5]:
from torch import nn

# 텍스트 분류 모델 정의 (DistilBERT + Linear layer)
class TextClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        # 사전학습된 DistilBERT 모델을 encoder로 불러옴 (pretrained transformer)
        self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')


        # [CLS] 토큰 분류기 정의 => Binary Classification
        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        # encoder에 input_ids 전달
        x = self.encoder(x)['last_hidden_state']

        # [CLS] 토큰 위치 벡터를 classification head에 전달
        x = self.classifier(x[:, 0])

        return x  # logit 출력 (ex: [0.3, -1.2, 1.5, 0.7])

model = TextClassifier()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


### DistilBERT(encoder) freeze 처리
- `model.encoder`: DistilBERT 모델
- `requires_grad = False`: **역전파에서 gradient가 계산되지 않도록**처리
  - 즉, **pre-trained encoder는 freeze하고** classifer 부분만 학습하도록 설정

<br>

**freeze 처리 이유**   
- 학습 속도가 빨라짐
- 데이터셋이 작을 경우 **overfitting 방지**
- classifier (Linear layer)만 fine tuning

In [6]:
for param in model.encoder.parameters():
  param.requires_grad = False

### 학습
**optimizer**
- `Adam`   

**loss function**
- `BCEWithLogitsLoss()`: binary classification

**epoch**   
- 5   

> 매 배치마다 zero_grad -> forward -> loss -> backward -> step 반복

**`preds = model(inputs)[..., 0]`**   
- output shape이 [batch_size, 1]일 때 [batch_size,]로 reshape해줌

In [7]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

# 학습 설정
lr = 0.001
model = model.to('cuda')  # 모델을 GPU로 이동
loss_fn = nn.BCEWithLogitsLoss()  # 이진 분류용 손실 함수

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 5

# 학습 루프
for epoch in range(n_epochs):
    total_loss = 0.
    model.train()  # 학습 모드 설정

    for data in train_loader:
        model.zero_grad()  # 이전 gradient 초기화

        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda').float()  # GPU 이동 및 float 변환

        preds = model(inputs)[..., 0]  # 출력 차원 맞추기 (batch_size,)

        loss = loss_fn(preds, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트

        total_loss += loss.item()  # loss 누적

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch   0 | Train Loss: 3.7471729926764965
Epoch   1 | Train Loss: 0.5308846943080425
Epoch   2 | Train Loss: 0.23809954430907965
Epoch   3 | Train Loss: 0.16150250332430005
Epoch   4 | Train Loss: 0.12457623751834035


### 정확도 측정
**Threshold**: 임계값 처리   
> 모델이 출력한 연속적인 수치(logit or 확률)를 0 or 1로 변환하는 과정

- 주로 **binary classification**에서 사용
  - 보통 sigmoid -> threshold
  - Multi-class Classification에서는 softmax -> argmax로 함
- 모델이 출력한 값이 특정 임계값(보통 0.5)을 넘으면 **Positive (1)**
- or **Negative (0)**

In [8]:
def accuracy(model, dataloader):
    cnt = 0      # 전체 샘플 수
    acc = 0      # 정답 개수 누적

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        preds = model(inputs)  # 로짓(logit) 출력

        # 시그모이드는 생략 가능 (BCEWithLogitsLoss를 썼다면 threshold만 적용)
        # preds = torch.argmax(preds, dim=-1)
        preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]  # 총 샘플 수 누적
        acc += (labels == preds).sum().item()  # 예측이 맞은 수 누적

    return acc / cnt  # 정확도 반환

# 평가 시 gradient 계산 비활성화
with torch.no_grad():
    model.eval()  # 평가 모드로 전환 (계산 비활성화)
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)

    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")


=========> Train acc: 1.000 | Test acc: 1.000
